In [ ]:
import json
import re
import unicodedata
import os
from vncorenlp import VnCoreNLP

Loaded 3239 questions from Văn học
Loaded 670 questions from sử
Loaded 838 questions from địa
Loaded 72 questions from anh


In [ ]:
def fix_answer_format(ans):
    if not isinstance(ans, str):
        return ans
    # Thêm dấu chấm + space nếu thiếu sau A/B/C/D
    ans = re.sub(r"^([A-D])[\.\s]*", r"\1. ", ans.strip())
    # Bỏ khoảng trắng thừa
    ans = re.sub(r"\s+", " ", ans).strip()
    return ans


In [ ]:
def simple_tokenize(text):
    """
    Simple Vietnamese tokenization function
    Args:
        text (str): Input text to tokenize
    Returns:
        str: Tokenized text with spaces between words
    """
    if not isinstance(text, str):
        return ""
    
    # Tách từ đơn giản bằng khoảng trắng
    words = text.split()
    
    # Tách số và chữ
    processed_words = []
    for word in words:
        # Tách số và chữ
        word = re.sub(r'([A-Za-z])([0-9])', r'\1 \2', word)
        word = re.sub(r'([0-9])([A-Za-z])', r'\1 \2', word)
        # Tách chữ hoa
        word = re.sub(r'([a-z])([A-Z])', r'\1 \2', word)
        processed_words.append(word)
    
    return ' '.join(processed_words)

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Chuẩn hóa Unicode (xử lý dấu tiếng Việt)
    text = unicodedata.normalize("NFC", text)

    # Bỏ xuống dòng, tab, khoảng trắng thừa
    text = re.sub(r"\s+", " ", text)

    # Bỏ ký tự đặc biệt không cần thiết
    text = re.sub(r"[^\w\sÀ-ỹ]", "", text)

    return text.strip()

In [ ]:
# def clean_question(qa_pairs, all_data):
#     for subject, data in all_data.items():
#         for item in data:
#             if "question" in item and "answers" in item:
#                 clean_q = clean_text(item["question"])
#                 # Xử lý đáp án: vừa clean_text vừa fix format
#                 clean_ans = [fix_answer_format(clean_text(ans)) for ans in item["answers"]]
#                 correct_ans = item.get("correct_answer")
#                 explanation = clean_text(item.get("explanation", ""))

#                 if clean_q and clean_ans:  # chỉ lấy nếu có cả câu hỏi và câu trả lời
#                     qa_pairs.append({
#                         "subject": subject,
#                         "question": clean_q,
#                         "answers": clean_ans,
#                         "correct_answer": correct_ans,
#                         "explanation": explanation if explanation else None
#                     })

def clean_question(qa_pairs, all_data):
    for subject, data in all_data.items():
        for idx, item in enumerate(data):
            # Nếu thiếu field bắt buộc thì bỏ qua
            if not item.get("question") or not item.get("answers"):
                print(f"Bỏ câu {idx} trong {subject}: thiếu question/answers")
                continue  

            # Nếu bất kỳ đáp án nào là None thì bỏ qua luôn
            if any(ans is None for ans in item["answers"]):
                print(f"Bỏ câu {idx} trong {subject}: có đáp án null")
                continue  

            try:
                clean_q = clean_text(item["question"])
                clean_ans = [fix_answer_format(clean_text(str(ans))) for ans in item["answers"]]

                # Lấy correct answer
                correct_ans_label = item.get("correct_answer")
                correct_ans_full = None
                if correct_ans_label:
                    correct_ans_label = str(correct_ans_label).strip().upper()
                    # Ánh xạ A/B/C/D -> đáp án
                    label_map = {"A": 0, "B": 1, "C": 2, "D": 3}
                    if correct_ans_label in label_map and label_map[correct_ans_label] < len(clean_ans):
                        ans_text = clean_ans[label_map[correct_ans_label]]
                        # Nếu ans_text đã bắt đầu bằng "B. " thì giữ nguyên,
                        # nếu chưa thì thêm "B. " vào trước
                        if re.match(rf"^{correct_ans_label}\.\s", ans_text):
                            correct_ans_full = ans_text
                        else:
                            correct_ans_full = f"{correct_ans_label}. {ans_text}"
                    else:
                        correct_ans_full = correct_ans_label

                explanation = clean_text(item.get("explanation", ""))

                qa_pairs.append({
                    "subject": subject,
                    "question": clean_q,
                    "answers": clean_ans,
                    "correct_answer": correct_ans_full,
                    "explanation": explanation if explanation else None
                })
            except Exception as e:
                print(f"Lỗi khi xử lý câu {idx} trong {subject}: {e}")
                continue





In [ ]:
# Tách từ

def clean_word(processed_qa, qa_pairs):
    for qa in qa_pairs:
        processed_qa.append({
            "subject": qa["subject"],
            "question": simple_tokenize(qa["question"]),
            "answers": [simple_tokenize(ans) for ans in qa["answers"]],
            "correct_answer": qa["correct_answer"],
            "explanation": simple_tokenize(qa["explanation"]) if qa["explanation"] else None
        })

In [ ]:
def run_data_processed(processed_qa):
    # Lưu kết quả vào file
    if not os.path.exists("data_processed"):
        os.makedirs("data_processed")

    questions_by_subject = {}
    for qa in processed_qa:
        subject = qa['subject']
        if subject not in questions_by_subject:
            questions_by_subject[subject] = []
        questions_by_subject[subject].append(qa)

    for subject, questions in questions_by_subject.items():
        output_file = f"data_processed/{subject}_processed.txt"
        
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(f"Tổng số câu hỏi môn {subject}: {len(questions)}\n\n")
            
            for i, qa in enumerate(questions, 1):
                f.write(f"Câu {i}:\n")
                f.write(f"Môn: {qa['subject']}\n")
                f.write(f"Câu hỏi: {qa['question']}\n")
                f.write("Các đáp án:\n")
                for j, ans in enumerate(qa['answers'], 1):
                    f.write(f"  {ans}\n")
                f.write(f"Đáp án đúng: {qa['correct_answer']}\n")
                if qa['explanation']:
                    f.write(f"Giải thích: {qa['explanation']}\n")
                f.write("\n" + "-"*50 + "\n\n")
        
        print(f"\nĐã xử lý và lưu {len(questions)} câu hỏi môn {subject} vào file '{output_file}'")